# Q1

In [1]:
from bs4 import BeautifulSoup
import urllib.request
# import necessary libraries which we used in class to do web crawling

In [2]:
seed_url = 'https://press.un.org/en'
all_url = [seed_url]
seen = [seed_url]
opened = []
release = []

while len(all_url)>0 and len(release) <10:
    curr_url = all_url.pop(0) # take out the first url
    try:
        print(f'Current Url: {curr_url}')
        req = urllib.request.Request(curr_url,headers={'User-Agent': 'Mozilla/5.0'}) # we crawl the same type of page as we did in class
        webpage = urllib.request.urlopen(req).read()
        opened.append(curr_url)
        
    except:
        print(f'Unable To Access: {curr_url}')
        continue
    
    soup = BeautifulSoup(webpage) # arrange in nice soup format
    
    for tag in soup.find_all('a', href = True): # find all urls
        childUrl = tag['href'] # get this url
        o_childurl = childUrl # save the original o_childurl
        childUrl = urllib.parse.urljoin(seed_url, childUrl)

        if seed_url in childUrl and childUrl not in seen:
            print(f'Get A New childurl: {childUrl}')
            all_url.append(childUrl)
            seen.append(childUrl)
        else:
            print(f'Abandoning url: {childUrl}')
    
    # see if curr_url is a press release
    press_release_tag = soup.find('a', href='/en/press-release', hreflang='en',string='Press Release')
    
    if press_release_tag != None:
        print(f'Current Url {curr_url} is a PRESS RELEASE!')
        release.append(curr_url)

print('Here are the first 10 press release urls found!')
for release_url in release:
    print(release_url)
    

Current Url: https://press.un.org/en
Get A New childurl: https://press.un.org/en#main-content
Abandoning url: https://press.un.org/en
Abandoning url: https://www.un.org/en
Abandoning url: https://press.un.org/fr/home
Abandoning url: https://www.un.org/en
Abandoning url: https://press.un.org/en
Abandoning url: https://press.un.org/en
Get A New childurl: https://press.un.org/en/content/secretary-general
Abandoning url: https://press.un.org/en/content/secretary-general
Get A New childurl: https://press.un.org/en/content/secretary-general/press-release
Get A New childurl: https://press.un.org/en/content/secretary-general/press-conference
Get A New childurl: https://press.un.org/en/content/general-assembly
Abandoning url: https://press.un.org/en/content/general-assembly
Get A New childurl: https://press.un.org/en/content/general-assembly/meetings-coverage
Get A New childurl: https://press.un.org/en/content/general-assembly/press-release
Get A New childurl: https://press.un.org/en/content/se

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Abandoning url: https://press.un.org/en#main-content
Abandoning url: https://press.un.org/en
Abandoning url: https://www.un.org/en
Abandoning url: https://www.un.org/en
Abandoning url: https://press.un.org/en
Abandoning url: https://press.un.org/en
Abandoning url: https://press.un.org/en/content/secretary-general
Abandoning url: https://press.un.org/en/content/secretary-general
Abandoning url: https://press.un.org/en/content/secretary-general/press-release
Abandoning url: https://press.un.org/en/content/secretary-general/press-conference
Abandoning url: https://press.un.org/en/content/general-assembly
Abandoning url: https://press.un.org/en/content/general-assembly
Abandoning url: https://press.un.org/en/content/general-assembly/meetings-coverage
Abandoning url: https://press.un.org/en/content/general-assembly/press-release
Abandoning url: https://press.un.org/en/content/security-council
Abandoning url: https://press.un.org/en/content/security-council
Abandoning url: https://press.un.o

In [3]:
release

['https://press.un.org/en/2023/sc15431.doc.htm',
 'https://press.un.org/en/2023/sc15430.doc.htm',
 'https://press.un.org/en/2023/sc15429.doc.htm',
 'https://press.un.org/en/2023/sgsm21964.doc.htm',
 'https://press.un.org/en/2023/sgsm21963.doc.htm',
 'https://press.un.org/en/2023/sgsm21962.doc.htm',
 'https://press.un.org/en/2023/sgsm21960.doc.htm',
 'https://press.un.org/en/2023/sgsm21961.doc.htm',
 'https://press.un.org/en/2023/sgsm21959.doc.htm',
 'https://press.un.org/en/2023/sgsm21958.doc.htm']

# Q2

In [ ]:
from tqdm import tqdm

seed_url = 'https://www.europarl.europa.eu/news/en/press-room'
all_url = [seed_url]
seen = [seed_url]
opened = []
release = []

pbar = tqdm(desc="Scraping URLs", unit="URL", position=0, leave=True)

while all_url and len(release) < 10:
    all_url = sorted(all_url, key=lambda s: 'crisis' not in s.lower()) # sort the urls placing those more likely to include crisis in front
    
    curr_url = all_url.pop(0)
    
    try:
        req = urllib.request.Request(curr_url, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urllib.request.urlopen(req).read()
        opened.append(curr_url)
    except:
        continue

    soup = BeautifulSoup(webpage, 'html.parser')
    
    for tag in soup.find_all('a', href=True):
        childUrl = tag['href']
        childUrl = urllib.parse.urljoin(seed_url, childUrl)

        if seed_url in childUrl and childUrl not in seen:
            all_url.append(childUrl)
            seen.append(childUrl)
    
    plenary_session_tag = soup.find('span', class_="ep_name", string='Plenary session')
    crisis_title_tag = any('crisis' in title_tag.text.lower() for title_tag in soup.find_all('title'))
    
    if plenary_session_tag and crisis_title_tag:
        release.append(curr_url)
        
    pbar.set_description(f"all_url: {len(all_url)}: [{all_url[:5]}] , seen: {len(seen)}, release: {len(release)}, opened: {len(opened)}")
    pbar.update(1)  # Manually update the progress bar by 1

pbar.close()

print('Here are the first 3 press release about plenary session about crisis URLs found!')
for release_url in release:
    print(release_url)


all_url: 903: [['https://www.europarl.europa.eu/news/en/press-room/20210910IPR11915/new-eu-blue-card-rules-for-highly-qualified-immigrants-wishing-to-work-in-europe', 'https://www.europarl.europa.eu/news/en/press-room/20230320IPR77911/meps-back-an-improved-single-work-and-residence-permit-for-non-eu-nationals', 'https://www.europarl.europa.eu/news/en/press-room/20230707IPR02437/search-and-rescue-meps-demand-more-eu-action-to-save-lives-at-sea', 'https://www.europarl.europa.eu/news/en/press-room/20210702IPR07622/high-time-for-council-and-commission-to-protect-europeans-and-the-eu-meps-say', 'https://www.europarl.europa.eu/news/en/press-room/20210430IPR03214/gender-equality-parliament-strives-to-be-frontrunner-among-eu-institutions']] , seen: 1215, release: 2, opened: 278: : 278URL [05:59,  1.65s/URL]                                                                                                                                                                                              